# Projeto 5 - Classificação binária breast - carregando classificador

## 1. Importando bibliotecas

In [19]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import accuracy_score
torch.__version__

'2.5.0'

In [2]:
torch.set_default_device("mps")
device = torch.device("mps")

# torch.set_default_device("cpu")
# device = torch.device("cpu")

In [3]:
print(torch.backends.mps.is_available())  # Deve retornar True
print(torch.backends.mps.is_built())  # Deve retornar True

True
True


## 2. Carregamento do classificador

In [4]:
np.random.seed(123)
torch.manual_seed(123)

In [5]:
class classificador_torch(nn.Module):
    def __init__(self):
        super().__init__()

        # 30 -> 16 -> 16 -> 1
        self.dense0 = nn.Linear(30, 8)
        torch.nn.init.normal_(self.dense0.weight, mean=0.0, std=0.05)

        self.dense1 = nn.Linear(8, 8)
        torch.nn.init.normal_(self.dense1.weight, mean=0.0, std=0.05)

        self.dense2 = nn.Linear(8, 1)
        torch.nn.init.normal_(self.dense2.weight, mean=0.0, std=0.05)
        
        self.activation = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        
        self.output = nn.Sigmoid()

    def forward(self, X):
        X = self.dense0(X)
        X = self.activation(X)
        X = self.dropout(X)
        
        X = self.dense1(X)
        X = self.activation(X)
        X = self.dropout(X)
        
        X = self.dense2(X)
        X = self.activation(X)
        X = self.dropout(X)

        X = self.output(X)
        return X

In [6]:
classificador = classificador_torch()

In [7]:
state_dict = torch.load('models/classificador.pth')
state_dict

/var/folders/rb/g2rznbp50d56w_26l6lfc72c0000gn/T/ipykernel_24385/636142537.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load('models/classificador.

OrderedDict([('dense0.weight',
              tensor([[ 7.4574e-02, -1.7704e-02, -2.2785e-03,  5.7946e-03, -1.6917e-02,
                        9.6920e-03, -1.9103e-02, -3.1056e-02, -3.1934e-03,  1.5530e-04,
                       -1.0100e-02, -2.0573e-02, -5.6476e-03,  1.8085e-04,  1.1633e-33,
                        1.5585e-08, -1.6096e-04, -9.8718e-34,  1.3048e-08,  3.1394e-11,
                        2.7221e-02, -3.1077e-02, -8.1115e-02, -6.0618e-02,  8.6933e-02,
                       -3.1037e-02, -1.7792e-02, -8.7776e-03, -8.0101e-02, -3.9559e-03],
                      [ 4.4589e-02,  2.1079e-02,  4.5174e-03, -3.9000e-02,  5.9652e-09,
                        4.0631e-03,  2.2769e-04, -2.4859e-05,  1.6553e-02,  9.5434e-08,
                       -4.6133e-03,  5.6078e-04, -1.4856e-02,  1.1423e-01, -2.3690e-19,
                       -2.6677e-18, -7.9293e-04,  5.5048e-19, -2.8171e-18,  1.6550e-19,
                       -5.7186e-03,  3.4836e-02, -4.0424e-02,  1.5331e-03,  2.1007e-02,


In [8]:
classificador.load_state_dict(state_dict)

<All keys matched successfully>

## 3. Previsões

In [10]:
novo_registro = torch.tensor([[15.80, 8.34, 118, 900, 0.10, 0.26, 0.08, 0.134, 0.178,
                  0.20, 0.05, 1098, 0.87, 4500, 145.2, 0.005, 0.04, 0.05, 0.015,
                  0.03, 0.007, 23.15, 16.64, 178.5, 2018, 0.14, 0.185,
                  0.84, 158, 0.363]], dtype = torch.float)

In [12]:
classificador.eval()
previsao = classificador(novo_registro)
previsao = (previsao.detach().cpu().numpy() > 0.5)
previsao

array([[ True]])

In [14]:
previsores = pd.read_csv('data/entradas_breast.csv')
classe = pd.read_csv('data/saidas_breast.csv')

In [15]:
previsores = torch.tensor(np.array(previsores), dtype=torch.float)
classe = torch.tensor(np.array(classe), dtype=torch.float)

In [16]:
previsoes = classificador.forward(previsores).detach()

In [17]:
previsoes

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5345],
        [0.7760],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.6592],
        [0.7178],
        [1.0000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5099],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [1.0000],
        [0.5000],
        [0.5117],
        [0.5484],
        [0.9542],
        [0.5000],
        [0.5000],
        [0.6487],
        [0.5000],
        [1.0000],
        [0.5511],
        [0.6948],
        [0.7246],
        [0.7178],
        [0.7178],
        [0.8323],
        [0.5000],
        [0.5000],
        [0

In [22]:
F.binary_cross_entropy(previsoes, classe).cpu().numpy()

array(0.4069249, dtype=float32)

In [23]:
accuracy_score(classe.cpu().numpy(), (previsoes.cpu() > 0.5).numpy())

0.929701230228471